In [21]:
import pandas as pd
import numpy as np
from dgutils.pandas import add_columns

In [2]:
df_dataset = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/data_processing/rna_cg/data/rfam.pkl')
df_pred = pd.read_pickle('/Users/alicegao/work/psi-lab-sandbox/rna_ss/tools/e2efold_2/e2efold_productive/tmp/rfam151.pkl')

In [5]:
df = pd.merge(df_dataset, df_pred, on='seq')

In [9]:
df.head()

,len,one_idx,seq,seq_id,pred_idx
0,117,"([0, 1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 16, 17, 1...",UUCGGUGGUUAUAGCGGUGGGGAAACACCCGGUCCCAUUCCGAACC...,RF00001_A,"[[0, 1, 2, 3, 4, 5, 6, 7, 13, 14, 15, 16, 17, ..."
1,150,"([32, 33, 35, 36, 43, 44, 45, 46, 47, 58, 59, ...",UAGCUUCAGCGAUGGAUCGGUUGCAUCGAGUAUCGAUGAAGAACGC...,RF00002_B,"[[2, 4, 13, 14, 17, 21, 22, 26, 29, 30, 33, 35..."
2,161,"([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, ...",AUACUUACCUGGACGGGGUCAAUGGAUGAUCUAUAAGGUCCAUGGC...,RF00003_B,"[[5, 14, 22, 35, 48, 49, 70, 75, 101, 119, 143..."
3,145,"([6, 7, 8, 10, 11, 12, 13, 46, 47, 48, 49, 50,...",AUAUCUUCUCGGCUAUUUAGCUAAGAUCAAGUUAUUAAACUGUUCU...,RF00004_A,"[[9, 12, 13, 19, 19, 19, 19, 19, 19, 19, 19, 1..."
4,72,"([0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ...",GGGCCCAUAGCUCAGUGGUAGAGUGCCUCCUUUGCAAGGAGGAUGC...,RF00005_A,"[[0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 25, 26, ..."


In [18]:
class EvalMetric(object):

#     @staticmethod
#     def _check_arr(arr):
#         assert len(arr.shape) == 2
#         assert arr.shape[0] == arr.shape[1]
#         assert np.all((arr == 0) | (arr == 1))
#         assert np.max(np.sum(arr, axis=0)) <= 1
#         assert np.max(np.sum(arr, axis=1)) <= 1

    @staticmethod
    def sensitivity(_pred, _target):
        # numerator: number of correct predicted base pairs
        # denominator: number of true base pairs
        assert _pred.shape[0] == _target.shape[0]
        n = _pred.shape[0]
        # set lower triangular to be all 0's
        pred = _pred.copy()
        target = _target.copy()
        pred[np.tril_indices(n)] = 0
        target[np.tril_indices(n)] = 0
        # checks
#         EvalMetric._check_arr(pred)
#         EvalMetric._check_arr(target)
        # metric
        idx_true_base_pair = np.where(target == 1)
        return float(np.sum(pred[idx_true_base_pair]))/np.sum(target)

    @staticmethod
    def ppv(_pred, _target):
        # numerator: number of correct predicted base pairs
        # denominator: number of predicted base pairs
        assert _pred.shape[0] == _target.shape[0]
        n = _pred.shape[0]
        # set lower triangular to be all 0's
        pred = _pred.copy()
        target = _target.copy()
        pred[np.tril_indices(n)] = 0
        target[np.tril_indices(n)] = 0
        # checks
#         EvalMetric._check_arr(pred)
#         EvalMetric._check_arr(target)
        # metric
        idx_predicted_base_pair = np.where(pred == 1)
        return float(np.sum(target[idx_predicted_base_pair])/np.sum(pred))

    @staticmethod
    def f_measure(sensitivity, ppv):
        return (2 * sensitivity * ppv)/(sensitivity + ppv)

In [19]:
eval_metric = EvalMetric()

In [22]:
def add_metrics(seq_len, one_idx, pred_idx):
    arr_target = np.zeros((seq_len, seq_len))
    arr_target[one_idx] = 1
    arr_pred = np.zeros((seq_len, seq_len))
    arr_pred[pred_idx] = 1

    sensitivity = eval_metric.sensitivity(arr_pred, arr_target)
    ppv = eval_metric.ppv(arr_pred, arr_target)
    f_measure = eval_metric.f_measure(sensitivity, ppv)
    
    return sensitivity, ppv, f_measure

In [23]:
df = add_columns(df, ['sensitivity', 'ppv', 'f_measure'], ['len', 'one_idx', 'pred_idx'], add_metrics)

/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """
/Users/alicegao/anaconda2/envs/gk_mz_plot/lib/python2.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in double_scalars


In [24]:
# for _, row in df.iterrows():
    
#     seq_len = row['len']
#     one_idx = row['one_idx']
#     pred_idx = row['pred_idx']

#     print(row['seq'])
    
#     arr_target = np.zeros((seq_len, seq_len))
#     arr_target[one_idx] = 1
#     arr_pred = np.zeros((seq_len, seq_len))
#     arr_pred[pred_idx] = 1

#     sensitivity = eval_metric.sensitivity(arr_pred, arr_target)
#     ppv = eval_metric.ppv(arr_pred, arr_target)
#     f_measure = eval_metric.f_measure(sensitivity, ppv)

#     print(sensitivity, ppv, f_measure)

In [27]:
# df[['seq_id', 'len', 'sensitivity', 'ppv', 'f_measure', 'seq']]

In [28]:
df.describe()

,f_measure,len,ppv,sensitivity
count,75.000000,151.000000,151.000000,151.000000
mean,0.132609,136.298013,0.088511,0.059149
std,0.169848,102.036451,0.164590,0.136486
min,0.012158,23.000000,0.000000,0.000000
25%,0.041241,67.000000,0.000000,0.000000
50%,0.066667,104.000000,0.000000,0.000000
75%,0.147099,158.500000,0.102632,0.053813
max,1.000000,568.000000,1.000000,1.000000


In [29]:
# df.iloc[3].pred_idx  # suspicious?